In [3]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [2]:
import pandas as  pd

train = pd.read_csv("labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("testData.tsv",header=0,delimiter="\t",quoting=3)

In [3]:
from sklearn.cluster import KMeans
import time

start = time.time()

word_vector = model.wv.vectors
num_clusters = int(word_vector.shape[0]/5)

Kmeans_clustering = KMeans(n_clusters=int(num_clusters))
idx = Kmeans_clustering.fit_predict(word_vector)

end = time.time()
elapsed = end-start
print("time taken for K-means clustering:",elapsed,"seconds")

time taken for K-means clustering: 81.39464282989502 seconds


In [4]:
word_centroid_map = dict(zip(model.wv.index_to_key,idx))

In [5]:
for cluster in range(0,10):
    print("\nCluster %d"%cluster)
    words = []
    for k,v in word_centroid_map.items():
        if v==cluster:
            words.append(k)
    print(words)


Cluster 0
['owner', 'manager', 'keeper', 'janitor', 'cosmo', 'bartender', 'pa', 'nickname', 'patron', 'proprietor', 'dolphin', 'penthouse']

Cluster 1
['solar', 'signals']

Cluster 2
['roast', 'tanya']

Cluster 3
['amusingly', 'delirious', 'gloriously', 'ott', 'enjoyably', 'sporadically']

Cluster 4
['dates', 'trips', 'beds', 'showers', 'lingerie', 'bushes', 'rode', 'crushes']

Cluster 5
['preaching', 'communicating']

Cluster 6
['cinematic', 'filmmaking', 'cinematographic', 'filmic', 'grandiose', 'chiefly', 'reworking']

Cluster 7
['secret', 'building', 'lab', 'plant', 'facility', 'laboratory', 'chemical', 'engineering', 'plutonium', 'sanitarium']

Cluster 8
['claims', 'pretending', 'claiming', 'pretends', 'trusted']

Cluster 9
['penn', 'connery', 'bean', 'astin', 'bana']


In [6]:
import numpy as np
def create_bag_of_centroids(wordlist,word_centroid_map):
    num_centroids = max(word_centroid_map.values())+1
    bag_of_centroids = np.zeros(num_centroids,dtype="float32")
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word] 
            bag_of_centroids[index]+=1
    return bag_of_centroids

In [8]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import pandas as  pd

train = pd.read_csv("labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("testData.tsv",header=0,delimiter="\t",quoting=3)

def review_to_wordlist(review,remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    words = review_text.lower().split()
    if remove_stopwords==True:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return words

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True))

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True))

/var/folders/9c/fs4ghgt579d29fb6jsxdrls40000gn/T/ipykernel_27698/3633448914.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


In [9]:
train_centroids =  np.zeros((train['review'].size,int(num_clusters)),dtype="float32")
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review,word_centroid_map)
    counter+=1

test_centroids =  np.zeros((test['review'].size,int(num_clusters)),dtype="float32")
counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review,word_centroid_map)
    counter+=1

In [11]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data={"Id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
